## 生成对抗网络 (Generative Adversarial Networks, GAN)


### 导入模块

In [1]:
import torch
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from torch import nn
from torch.autograd import Variable
from torch import optim
import os

### 加载数据

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
 
mnist = torchvision.datasets.MNIST(root="/Users/cynthiashi/Documents/DATA", 
                                   train=True, 
                                   download=True, 
                                   transform=transform)
dataloader = torch.utils.data.DataLoader(mnist, batch_size=100, shuffle=True)

### 定义判别器网络

In [8]:
class Dnet(nn.Module):
    def __init__(self):
        super(Dnet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=2),  # batch, 6, 30,30
            nn.LeakyReLU(0.2, True),
            nn.MaxPool2d(2, stride=2),  # batch, 6, 15, 15
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 12, 3, padding=2),  # batch, 12, 17, 17
            nn.LeakyReLU(0.2, True),
            nn.MaxPool2d(2, stride=2)  # batch, 12, 8, 8
        )
        self.fc = nn.Sequential(
            nn.Linear(12 * 8 * 8, 1024),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )
    # x.shape:[100,1,28,28]
    def forward(self, x):
        '''
        x: batch, width, height, channel=1
        '''
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)               # 将第二次卷积的输出拉伸为一行
        x = self.fc(x)
        x = x.squeeze(-1) # x.shape:[100,]
        return x

### 定义生成器网络

In [ ]:
class Gnet(nn.Module):
    def __init__(self):
        super(Gnet, self).__init__()
        self.fc = nn.Linear(128, 784)  # batch, 1,28,28
        self.br = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.ReLU(True)
        )
        self.downsample1 = nn.Sequential(
            nn.Conv2d(1, 12, 3, stride=1, padding=1),  # batch, 12, 28, 28
            nn.BatchNorm2d(12),
            nn.ReLU(True)
        )
        self.downsample2 = nn.Sequential(
            nn.Conv2d(12, 6, 3, stride=1, padding=1),  # batch, 6, 28, 28
            nn.BatchNorm2d(6),
            nn.ReLU(True)
        )
        self.downsample3 = nn.Sequential(
            nn.Conv2d(6, 1, 3, stride=1, padding=1),  # batch, 1, 28, 28
            nn.Tanh()
        )
    # x.shape:[100,128]
    def forward(self, x):
        x = self.fc(x) # # x.shape:[100,784]
        x = x.view(x.size(0), 1, 28, 28) # x.shape:[100,1,28,28]
        x = self.br(x) # x.shape:[100,1,28,28]
        x = self.downsample1(x) # x.shape:[100,12,28,28]
        x = self.downsample2(x) # x.shape:[100,6,28,28]
        x = self.downsample3(x) # x.shape:[100,1,28,28]
        return x

### 定义向量转图像函数

In [ ]:
def to_img(x):
    y = (x + 1) * 0.5
    y = y.clamp(0, 1)
    # clamp（最小值，最大值）
    # 参数是值要钳制在范围内的属性或变量。
    # 如果参数位于最小数值和最大数值之间的数值范围内，则该函数将返回参数值。
    # 如果参数大于范围，该函数将返回最大数值。
    # 如果参数小于范围，该函数将返回最小数值。
    y = y.view(-1, 1, 28, 28)
    return y

### 定义GAN网络

In [ ]:
class Net:
    def __init__(self):
        self.dnet = Dnet()
        self.gnet = Gnet()
#         self.dnet = self.dnet.cuda()
#         self.gnet = self.gnet.cuda()
        self.Loss = nn.BCELoss()
        self.d_optimizer = optim.Adam(self.dnet.parameters(), lr=0.0002)
        self.g_optimizer = optim.Adam(self.gnet.parameters(), lr=0.0002)
 
    def forward(self, real_x, fack_x):
        self.real_d_out = self.dnet(real_x) # 将真样本输入到判别器，得到判别结果self.real_d_out
        g_out = self.gnet(fack_x) # 将噪声输入生成器产生假样本，g_out.shape:[100,1,28,28]
        self.g_d_out = net.dnet(g_out.detach()) # 将假样本输入判别器，得到判别结果self.g_d_out
 
    def backward(self, pos_y, nega_y, fack_xs):
        # 以下几行的目地是训练判别器，使得判别器遇到真实样本就给1，遇到假样本就给0；
        d_out_loss = self.Loss(self.real_d_out, pos_y) # 将真样本判别结果self.real_d_out和真实标签求loss
        g_d_loss = self.Loss(self.g_d_out, nega_y) # 将假样本判别结果self.g_d_out和假标签求loss
        self.d_loss = d_out_loss + g_d_loss
        self.d_optimizer.zero_grad()
        self.d_loss.backward(retain_graph = True)
        self.d_optimizer.step()
 
        # 以下几行的目的是训练生成器，使得生成器产生的假样本越来越接近真实；
        self.fack_g_out = self.gnet(fack_xs) # 将噪声输入生成器产生假样本
        self.fack_g_d_out = self.dnet(self.fack_g_out) #将假样本输入判别器，得到判别结果self.fack_g_d_out
        self.g_loss = self.Loss(self.fack_g_d_out, pos_y) # 将假样本的判别结果与正标签对比求loss,意思是让假样本越来越接近真实；
        self.g_optimizer.zero_grad()
        self.g_loss.backward()
        self.g_optimizer.step()

## 训练测试 

In [9]:
if __name__ == '__main__':
    if not os.path.exists('img'):
        os.mkdir('img')
    net = Net()
    for i in range(100):
        for x, y in dataloader:
            # x = x.view(x.size(0),-1)
            real_x = Variable(x) # x [100,1,28,28] ,real_x:真样本输入
            fack_x = Variable(torch.randn(100, 128)) #fack_x：噪声判别器训练  torch.randn(100, 128)标准正态分布
            pos_y = Variable(torch.ones(100)) # 真样本标签
            nega_y = Variable(torch.zeros(100)) # 假样本标签
            fack_xs = Variable(torch.randn(100, 128))# fack_xs：噪声生成器训练
            net.forward(real_x, fack_x) # 前向推理，输入real_x, fack_x，即真样本输入和噪声
            net.backward(pos_y, nega_y, fack_xs) #反向传播过程
            img = to_img(net.fack_g_out.data)
            D_Accuracy = ((net.real_d_out.mean() + 1 - net.fack_g_d_out.mean()) / 2).item()
            print(net.d_loss.item(), net.g_loss.item(), D_Accuracy)
            save_image(img, './img/fake_images-{}.png'.format(i + 1))

1.397040605545044 0.7234911918640137 0.5023056864738464
1.3566298484802246 0.7431190609931946 0.5124354958534241
1.3154358863830566 0.7616558074951172 0.5225348472595215
1.2750446796417236 0.7821409702301025 0.5334157347679138
1.2334818840026855 0.8007785677909851 0.5441862344741821
1.1953494548797607 0.821645975112915 0.554772675037384
1.1565848588943481 0.8458109498023987 0.5659701824188232
1.1116368770599365 0.868228018283844 0.5794936418533325
1.0696076154708862 0.8938953280448914 0.5917494893074036
1.0251731872558594 0.9217650890350342 0.6051967740058899
0.9814608097076416 0.9517571330070496 0.6187740564346313
0.9403311610221863 0.9870021343231201 0.6324034929275513
0.8903439044952393 1.0190619230270386 0.648471474647522
0.846778929233551 1.0535998344421387 0.662093997001648
0.801214337348938 1.1020358800888062 0.677667498588562
0.756084680557251 1.1565653085708618 0.6954687237739563
0.713951051235199 1.2133116722106934 0.7102813124656677
0.6578121781349182 1.271918773651123 0.729

0.012195209041237831 5.22744083404541 0.9944194555282593
0.010436861775815487 5.1605706214904785 0.9952333569526672
0.01083820778876543 5.199349403381348 0.9946594834327698
0.010694277472794056 5.27456521987915 0.9951453804969788
0.011007258668541908 5.390182971954346 0.9956519603729248
0.009649193845689297 5.4514479637146 0.9956803917884827
0.009826931171119213 5.552669048309326 0.9957482218742371
0.01034065056592226 5.590400218963623 0.995180070400238
0.012900743633508682 5.6199235916137695 0.9940356612205505
0.012993361800909042 5.525661468505859 0.9939187169075012
0.008680475875735283 5.474398136138916 0.9958387017250061
0.009831304661929607 5.445053577423096 0.995452344417572
0.008569348603487015 5.415005683898926 0.9958178400993347
0.010206744074821472 5.387896537780762 0.9949870109558105
0.011307812295854092 5.3767781257629395 0.9948194026947021
0.01343637052923441 5.266585826873779 0.9933322072029114
0.010867202654480934 5.304959774017334 0.9951805472373962
0.011182501912117004

0.009573287330567837 5.781294345855713 0.995413601398468
0.018804529681801796 5.4812469482421875 0.9924522042274475
0.013233870267868042 5.239606857299805 0.9932568669319153
0.012141566723585129 5.439335823059082 0.9956101775169373
0.011145438067615032 5.736660003662109 0.9959485530853271
0.00890201423317194 5.8314313888549805 0.9961969256401062
0.01180759072303772 6.167531967163086 0.9946286678314209
0.016609838232398033 5.8539276123046875 0.9917974472045898
0.015182901173830032 5.220749378204346 0.9912132620811462
0.009987139143049717 5.013244152069092 0.9943006634712219
0.012765795923769474 5.162590980529785 0.995046079158783
0.012697426602244377 5.317108631134033 0.9937978982925415
0.00900852307677269 5.856491565704346 0.9967249035835266
0.00752683961763978 6.154752254486084 0.9971190690994263
0.017013512551784515 6.143682479858398 0.9922862648963928
0.011543438769876957 5.913261890411377 0.993908166885376
0.006647716276347637 5.698575496673584 0.9963527321815491
0.0086646704003214

0.12182064354419708 4.394505023956299 0.9375781416893005
0.13673260807991028 3.8059241771698 0.9338851571083069
0.11604833602905273 4.642796516418457 0.959112823009491
0.18240845203399658 5.777258396148682 0.9655145406723022
0.14719372987747192 4.682188510894775 0.9398073554039001
0.1661854237318039 4.768764495849609 0.9547425508499146
0.1423836052417755 4.570804119110107 0.9494132995605469
0.12287013977766037 4.773838996887207 0.95294588804245
0.2161785066127777 3.7131292819976807 0.904926061630249
0.18259908258914948 4.053757667541504 0.959682047367096
0.13458368182182312 4.5354485511779785 0.9535472989082336
0.11527104675769806 4.316540718078613 0.942425549030304
0.13353118300437927 4.050944805145264 0.9439473152160645
0.15560804307460785 4.365564346313477 0.9366142153739929
0.09094167500734329 4.4373931884765625 0.9521179795265198
0.10914850234985352 4.9212751388549805 0.9534691572189331
0.15766140818595886 4.660165786743164 0.9465868473052979
0.16303551197052002 4.200168609619141 

0.18222934007644653 3.2585268020629883 0.9021792411804199
0.2180686593055725 3.923793077468872 0.9441032409667969
0.2202978879213333 4.427188396453857 0.9357085824012756
0.19445276260375977 4.343536376953125 0.9300151467323303
0.2437247335910797 3.6465914249420166 0.9064912796020508
0.237461119890213 3.510976552963257 0.9098590612411499
0.21930809319019318 3.639864206314087 0.9176169633865356
0.20351210236549377 4.549285888671875 0.9368969798088074
0.19009137153625488 4.126592636108398 0.9172192215919495
0.19995811581611633 3.156578302383423 0.8822483420372009
0.2941107451915741 3.5438506603240967 0.9141596555709839
0.2199147343635559 4.221785068511963 0.9330964088439941
0.24726516008377075 4.576051712036133 0.9297342300415039
0.2315337359905243 3.4233062267303467 0.8853800296783447
0.17328914999961853 3.1995420455932617 0.9033781886100769
0.2956555485725403 4.08285665512085 0.9412298798561096
0.23344960808753967 4.517695426940918 0.9265740513801575
0.2621663510799408 4.582769393920898

...

0.4081648290157318 2.572958469390869 0.8559661507606506
0.3992440700531006 2.919334650039673 0.8842514753341675
0.34124845266342163 3.1607730388641357 0.8880331516265869
0.32920700311660767 3.437110185623169 0.9087194800376892
0.4772621989250183 3.5393335819244385 0.8806225657463074
0.4084055423736572 3.3287301063537598 0.8719772100448608
0.5437622666358948 3.007981777191162 0.8188945055007935
0.35694000124931335 2.3342745304107666 0.8220567107200623
0.5596308708190918 2.8864352703094482 0.8725818395614624
0.413441002368927 2.9599664211273193 0.8765260577201843
0.36011168360710144 3.5758605003356934 0.8831727504730225
0.4817952513694763 3.5356149673461914 0.8623577356338501
0.5243691802024841 3.1963281631469727 0.840501070022583
0.44670602679252625 2.592028856277466 0.8438446521759033
0.3440287113189697 2.404639959335327 0.8448347449302673
0.4813118577003479 2.7935638427734375 0.8556786179542542
0.5116228461265564 2.8653500080108643 0.8377812504768372
0.4311959147453308 3.3981473445892

0.5281980037689209 2.6090850830078125 0.8416051864624023
0.4086577296257019 2.8170177936553955 0.8573259115219116
0.3252769410610199 3.2395401000976562 0.8864231705665588
0.4151056408882141 3.0252554416656494 0.8572832942008972
0.3244696855545044 2.9821605682373047 0.8761173486709595
0.37545230984687805 2.7536232471466064 0.8439876437187195
0.3291667401790619 2.4680488109588623 0.8570572733879089
0.46206939220428467 2.709401845932007 0.8451260328292847
0.44459640979766846 2.878958225250244 0.8792623281478882
0.45757126808166504 3.355694055557251 0.8731939792633057
0.40857255458831787 3.645542621612549 0.8850854635238647
0.43856537342071533 3.1587862968444824 0.8547618389129639
0.385117769241333 2.8405721187591553 0.8423625826835632
0.3838963806629181 2.488227367401123 0.8544744849205017
0.33587646484375 2.903688907623291 0.8865520358085632
0.3152182102203369 2.8219430446624756 0.8861337304115295
0.38968735933303833 2.9730162620544434 0.8460808992385864
0.4890507757663727 2.895963191986

0.32816195487976074 2.7275359630584717 0.8660120368003845
0.386343389749527 2.9123008251190186 0.8780168294906616
0.4486924409866333 2.815415620803833 0.8451682329177856
0.39088404178619385 3.2462410926818848 0.8926799297332764
0.4701307415962219 3.0470027923583984 0.8611863851547241
0.4403448700904846 3.152453660964966 0.8594349026679993
0.4610257148742676 2.4664711952209473 0.8141039609909058
0.42544639110565186 2.216038465499878 0.8367276787757874
0.4947991967201233 2.466773748397827 0.8410776257514954
0.5198633074760437 2.977325439453125 0.8714833855628967
0.37896230816841125 3.2751991748809814 0.8893255591392517
0.38194188475608826 3.4898581504821777 0.8803586959838867
0.31029796600341797 3.024460554122925 0.8770272731781006
0.4012330174446106 2.612489700317383 0.8388330936431885
0.45334821939468384 2.125542640686035 0.8072798848152161
0.31252965331077576 2.264763832092285 0.8516652584075928
0.33994045853614807 2.514741897583008 0.870614767074585
0.41189610958099365 3.044668674468

0.4708980917930603 2.7229275703430176 0.8440004587173462
0.363612562417984 3.0980777740478516 0.8812079429626465
0.40247875452041626 3.1552817821502686 0.8497340083122253
0.3151281476020813 2.745678186416626 0.8713759183883667
0.3759974539279938 2.6095824241638184 0.8488654494285583
0.5226927995681763 3.0337462425231934 0.8675833344459534
0.4167153239250183 2.6451809406280518 0.8417730927467346
0.4710018038749695 2.6213436126708984 0.8338532447814941
0.3998299837112427 2.7123284339904785 0.8444178700447083
0.47146838903427124 2.6499032974243164 0.8281607627868652
0.449817955493927 3.2041943073272705 0.883534848690033
0.5084364414215088 3.496683120727539 0.877357006072998
0.5123293399810791 3.038083791732788 0.8443247675895691
0.5061928033828735 3.152350425720215 0.857929527759552
0.5625418424606323 2.5544490814208984 0.8422383069992065
0.4587098956108093 2.230228900909424 0.818722665309906
0.5122727155685425 2.8589115142822266 0.883529007434845
0.48659616708755493 2.898076057434082 0.8

KeyboardInterrupt: 